In [1]:
import os
import json
import py_vncorenlp
import pickle
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer


/opt/conda/envs/wiki310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
json_dir = "/workspace/Vi-VLM-TTDN/data/wiki_corpus/extracted"
embedding_model_name = "dangvantuan/vietnamese-embedding"
vncorenlp_path = "/workspace/Vi-VLM-TTDN/modules/vncorenlp"
chunk_size = 1
overlap = 0

In [3]:
embed_model = SentenceTransformer(embedding_model_name)
tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)
segmenter = py_vncorenlp.VnCoreNLP(save_dir=vncorenlp_path, annotators=["wseg"])

2025-04-01 14:57:28 INFO  WordSegmenter:24 - Loading Word Segmentation model


In [4]:
corpus = []
list_chunks = []
metadata = []
embeddings = []

In [5]:
# load all doc to corpus list
for root, dirs, files in os.walk(json_dir):
    for filename in tqdm(files):
        file_path = os.path.join(root, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                try:
                    data = json.loads(line)
                    if isinstance(data, dict):
                        corpus.append(data)
                except json.JSONDecodeError as e:
                    print(f"[WARNING] Skipping malformed line in file {filename} : {e}")

print(f"The keys in each article dictionary: {corpus[2].keys()}")
print(f"The number of articles: {len(corpus)}")

0it [00:00, ?it/s]
100%|██████████| 78/78 [00:00<00:00, 236.18it/s]

The keys in each article dictionary: dict_keys(['id', 'revid', 'url', 'title', 'text'])
The number of articles: 1591518


In [6]:
# Take 100 first particle to load
part_article = corpus[0:100]

In [7]:
part_article[2]

{'id': '4',
 'revid': '845147',
 'url': 'https://vi.wikipedia.org/wiki?curid=4',
 'title': 'Internet Society',
 'text': 'Internet Society hay ISOC là một tổ chức quốc tế hoạt động phi lợi nhuận, phi chính phủ và bao gồm các thành viên có trình độ chuyên ngành. Tổ chức này chú trọng đến: tiêu chuẩn, giáo dục và các vấn đề về chính sách. Với trên 145 tổ chức thành viên và 65.000 thành viên cá nhân, ISOC bao gồm những con người cụ thể trong cộng đồng Internet. Mọi chi tiết có thể tìm thấy tại website của ISOC.\nInternet Society nằm ở gần thủ đô Washington, DC, Hoa Kỳ và Geneva, Thụy Sĩ. Số hội viên của nó bao gồm hơn 145 tổ chức thành viên và hơn 65.000 cá nhân. Thành viên còn có thể tự lập một chi nhánh của tổ chức tùy theo vị trí hoặc sở thích. Hiện nay tổ chức có tới 90 chi nhánh trên toàn thế giới.\nNhiệm vụ và mục đích hoạt động.\nBảo đảm, cổ vũ cho sự phát triển, mở rộng và sử dụng Internet được thuận lợi nhất cho mọi người trên toàn thế giới.'}

In [8]:
list_segmented = segmenter.word_segment(part_article[4]['text'])
list_segmented

['Tiếng Việt hay tiếng Kinh là một ngôn_ngữ thuộc ngữ_hệ Nam_Á , được công_nhận là ngôn_ngữ chính_thức tại Việt_Nam .',
 'Đây là tiếng_mẹ_đẻ của khoảng 85% dân_cư Việt_Nam và nhiều người trong số 5 triệu Việt_kiều .',
 'Tiếng Việt còn là ngôn_ngữ thứ hai của các dân_tộc_thiểu_số tại Việt_Nam , và là một trong những ngôn_ngữ chính_thức của thành_phố San_Francisco , cũng là một ngôn_ngữ dân_tộc_thiểu_số được công_nhận tại Czechia và Slovakia .',
 'Dựa trên từ_vựng cơ_bản , tiếng Việt được phân_loại là một ngôn_ngữ thuộc ngữ_hệ Nam Á. Tiếng Việt là ngôn_ngữ có nhiều người nói nhất trong ngữ_hệ này ( nhiều hơn tổng_số người nói của tất_cả các ngôn_ngữ còn lại trong ngữ_hệ ) .',
 'Vì Việt_Nam thuộc Vùng văn_hoá Á_Đông , tiếng Việt cũng chịu nhiều ảnh_hưởng về từ tiếng Hán , do_vậy là ngôn_ngữ có ít điểm tương_đồng nhất với các ngôn_ngữ khác trong ngữ_hệ Nam Á. Lịch_sử .',
 'Theo A. G. Haudricourt giải_thích từ năm 1954 , nhóm ngôn_ngữ Việt-Mường ở thời_kỳ khoảng đầu Công_nguyên là những ngô

In [9]:
segmented_text = " ".join(list_segmented)
segmented_text

'Tiếng Việt hay tiếng Kinh là một ngôn_ngữ thuộc ngữ_hệ Nam_Á , được công_nhận là ngôn_ngữ chính_thức tại Việt_Nam . Đây là tiếng_mẹ_đẻ của khoảng 85% dân_cư Việt_Nam và nhiều người trong số 5 triệu Việt_kiều . Tiếng Việt còn là ngôn_ngữ thứ hai của các dân_tộc_thiểu_số tại Việt_Nam , và là một trong những ngôn_ngữ chính_thức của thành_phố San_Francisco , cũng là một ngôn_ngữ dân_tộc_thiểu_số được công_nhận tại Czechia và Slovakia . Dựa trên từ_vựng cơ_bản , tiếng Việt được phân_loại là một ngôn_ngữ thuộc ngữ_hệ Nam Á. Tiếng Việt là ngôn_ngữ có nhiều người nói nhất trong ngữ_hệ này ( nhiều hơn tổng_số người nói của tất_cả các ngôn_ngữ còn lại trong ngữ_hệ ) . Vì Việt_Nam thuộc Vùng văn_hoá Á_Đông , tiếng Việt cũng chịu nhiều ảnh_hưởng về từ tiếng Hán , do_vậy là ngôn_ngữ có ít điểm tương_đồng nhất với các ngôn_ngữ khác trong ngữ_hệ Nam Á. Lịch_sử . Theo A. G. Haudricourt giải_thích từ năm 1954 , nhóm ngôn_ngữ Việt-Mường ở thời_kỳ khoảng đầu Công_nguyên là những ngôn_ngữ hay phương_ngữ 

In [10]:
len(segmented_text)

26614

In [11]:
test_input_embedding = segmented_text[0:1200]

In [12]:
test_input_embedding

'Tiếng Việt hay tiếng Kinh là một ngôn_ngữ thuộc ngữ_hệ Nam_Á , được công_nhận là ngôn_ngữ chính_thức tại Việt_Nam . Đây là tiếng_mẹ_đẻ của khoảng 85% dân_cư Việt_Nam và nhiều người trong số 5 triệu Việt_kiều . Tiếng Việt còn là ngôn_ngữ thứ hai của các dân_tộc_thiểu_số tại Việt_Nam , và là một trong những ngôn_ngữ chính_thức của thành_phố San_Francisco , cũng là một ngôn_ngữ dân_tộc_thiểu_số được công_nhận tại Czechia và Slovakia . Dựa trên từ_vựng cơ_bản , tiếng Việt được phân_loại là một ngôn_ngữ thuộc ngữ_hệ Nam Á. Tiếng Việt là ngôn_ngữ có nhiều người nói nhất trong ngữ_hệ này ( nhiều hơn tổng_số người nói của tất_cả các ngôn_ngữ còn lại trong ngữ_hệ ) . Vì Việt_Nam thuộc Vùng văn_hoá Á_Đông , tiếng Việt cũng chịu nhiều ảnh_hưởng về từ tiếng Hán , do_vậy là ngôn_ngữ có ít điểm tương_đồng nhất với các ngôn_ngữ khác trong ngữ_hệ Nam Á. Lịch_sử . Theo A. G. Haudricourt giải_thích từ năm 1954 , nhóm ngôn_ngữ Việt-Mường ở thời_kỳ khoảng đầu Công_nguyên là những ngôn_ngữ hay phương_ngữ 

In [13]:
embedding_test = embed_model.encode(test_input_embedding)

In [14]:
embedding_test.shape

(768,)

In [15]:
test_tokens = embed_model.tokenizer(segmented_text)['input_ids']
len(test_tokens)

Token indices sequence length is longer than the specified maximum sequence length for this model (5627 > 512). Running this sequence through the model will result in indexing errors


5627

In [16]:
def count_tokens(text: str) -> int:
    return embed_model.tokenizer(text, return_attention_mask=False, return_token_type_ids=False)["input_ids"]

In [17]:
text = "Đây là một đoạn văn dùng để kiểm tra số lượng token."
num_tokens = count_tokens(segmented_text)
print(f"Số token: {len(num_tokens)}")

Số token: 5627


In [23]:
tokens_test_max = num_tokens[0:200]
print(tokens_test_max)

[0, 3606, 350, 118, 355, 8123, 8, 16, 2877, 187, 51649, 40825, 14068, 4, 11, 1803, 8, 2877, 495, 35, 56, 5, 282, 8, 32271, 7, 137, 9576, 1817, 56, 6, 36, 18, 12, 100, 173, 113, 6842, 5, 3606, 350, 50, 8, 2877, 129, 82, 7, 9, 4599, 35, 56, 4, 6, 8, 16, 12, 21, 2877, 495, 7, 214, 10979, 4, 32, 8, 16, 2877, 4599, 11, 1803, 35, 53493, 1224, 6, 10597, 5, 7731, 34, 25494, 1264, 4, 355, 350, 11, 4417, 8, 16, 2877, 187, 51649, 40825, 590, 3155, 3606, 350, 8, 2877, 10, 36, 18, 96, 67, 12, 51649, 40825, 23, 20, 36, 48, 1566, 18, 96, 7, 392, 9, 2877, 50, 44, 12, 51649, 40825, 19, 5, 1023, 56, 187, 4805, 472, 18198, 4, 355, 350, 32, 551, 36, 370, 28, 39, 355, 5463, 4, 6533, 8, 2877, 10, 343, 192, 6413, 67, 15, 9, 2877, 85, 12, 51649, 40825, 590, 3155, 3724, 5, 79, 3828, 8596, 27299, 13494, 29877, 1610, 39, 29, 7822, 4, 276, 2877, 16011, 11706, 25, 2055, 137, 127, 17504, 8, 21, 2877, 118, 35343, 17, 17366, 5055, 5, 674, 53, 4, 89, 473, 18915, 15, 1597, 5520, 6, 823]


In [24]:
decoded_text = tokenizer.decode(tokens_test_max, skip_special_tokens=True, clean_up_tokenization_spaces=True)
print(decoded_text)

Tiếng Việt hay tiếng Kinh là một ngôn_ngữ thuộc ngữ_hệ Nam_Á, được công_nhận là ngôn_ngữ chính_thức tại Việt_Nam. Đây là tiếng_mẹ_đẻ của khoảng 85% dân_cư Việt_Nam và nhiều người trong số 5 triệu Việt_kiều. Tiếng Việt còn là ngôn_ngữ thứ hai của các dân_tộc_thiểu_số tại Việt_Nam, và là một trong những ngôn_ngữ chính_thức của thành_phố San_Francisco, cũng là một ngôn_ngữ dân_tộc_thiểu_số được công_nhận tại Czechia và Slovakia. Dựa trên từ_vựng cơ_bản, tiếng Việt được phân_loại là một ngôn_ngữ thuộc ngữ_hệ Nam Á. Tiếng Việt là ngôn_ngữ có nhiều người nói nhất trong ngữ_hệ này ( nhiều hơn tổng_số người nói của tất_cả các ngôn_ngữ còn lại trong ngữ_hệ ). Vì Việt_Nam thuộc Vùng văn_hoá Á_Đông, tiếng Việt cũng chịu nhiều ảnh_hưởng về từ tiếng Hán, do_vậy là ngôn_ngữ có ít điểm tương_đồng nhất với các ngôn_ngữ khác trong ngữ_hệ Nam Á. Lịch_sử. Theo A. G. Haudricourt giải_thích từ năm 1954, nhóm ngôn_ngữ Việt-Mường ở thời_kỳ khoảng đầu Công_nguyên là những ngôn_ngữ hay phương_ngữ không thanh_đ

In [25]:
len(decoded_text)

1059

In [27]:
embed_model.encode(decoded_text).shape

(768,)

In [22]:
max_tokens = embed_model.tokenizer.model_max_length
max_tokens

512

In [28]:
print(embed_model._first_module().get_max_seq_length())

AttributeError: 'Transformer' object has no attribute 'get_max_seq_length'

In [34]:
import numpy as np

embeddings = np.load("/workspace/Vi-VLM-TTDN/outputs/wiki_embeddings.npy")
print("Embedding shape:", embeddings.shape)
print("Example vector:", embeddings[10])

Embedding shape: (3348, 768)
Example vector: [-0.012689    0.04984979 -0.09448653 -0.05139583  0.02033724  0.02731524
 -0.03086633 -0.008592   -0.00353245  0.01251643  0.01284153 -0.00239585
  0.04529267  0.01422104 -0.02002686  0.00121167 -0.0370364  -0.02075229
  0.02143185 -0.03612815  0.01424512 -0.03817632  0.03207061  0.02348634
 -0.05272267 -0.01615144  0.01974813  0.00436357 -0.04967909 -0.01293465
  0.06653659 -0.06066234  0.03962167 -0.02899079  0.04347555  0.00333516
 -0.01026396  0.00981466 -0.00095956 -0.09442191  0.02518732 -0.06972646
 -0.04616494 -0.03616979  0.06821566 -0.05062084  0.05881059 -0.0621549
  0.04914771  0.050658   -0.00746813  0.00967659  0.03747782  0.00851888
 -0.03828807 -0.02324267  0.01608338 -0.01510121  0.03953498  0.00025264
 -0.02060932 -0.00690729 -0.02024128  0.06713626  0.04862772 -0.07270028
 -0.01482407  0.00312529  0.00521963  0.01449692  0.05373411  0.01533218
  0.02056992 -0.00888302  0.03304089  0.01278367  0.00693141 -0.07311059
  0.005

In [33]:
import pickle

with open("/workspace/Vi-VLM-TTDN/outputs/wiki_metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

print("Metadata size:", len(metadata))
print("Example metadata:", metadata[10])

Metadata size: 3348
Example metadata: {'title': 'Tiếng Việt', 'doc_id': 4, 'chunk_id': 7, 'chunk_text': '", " tâm ", " minh ", " đức_", " thiên ", " tự_do "... giữ nguyên nghĩa chỉ khác cách đọc ; hay thay_đổi vị_trí như " nhiệt náo " thành " náo_nhiệt ", " thích phóng " thành " phóng_thích ", " đảm_bảo " thành " bảo_đảm "... ; hoặc rút_gọn như " thừa trần " thành " trần " ( trong " trần nhà " ), " lạc hoa sinh " thành " lạc " ( trong " củ lạc ", còn gọi là " đậu_phộng " )... ; hoặc đọc chệch đi như " sáp_nhập " ( chữ Hán : ) thành " sát_nhập ", "_thống kế'}


In [36]:
import json
with open("/workspace/Vi-VLM-TTDN/outputs/wiki_corpus.json", "r", encoding="utf-8") as f:
    corpus = json.load(f)

In [37]:
corpus[0]

[{'id': '1',
  'revid': '309098',
  'url': 'https://vi.wikipedia.org/wiki?curid=1',
  'title': 'Alexandre De Rhodes',
  'text': ''},
 {'id': '2',
  'revid': '501647',
  'url': 'https://vi.wikipedia.org/wiki?curid=2',
  'title': 'Trang Chính',
  'text': '&lt;templatestyles src="Wiki2021/styles.css" /&gt;__NOEDITSECTION__\n \n \n '},
 {'id': '4',
  'revid': '845147',
  'url': 'https://vi.wikipedia.org/wiki?curid=4',
  'title': 'Internet Society',
  'text': 'Internet Society hay ISOC là một tổ chức quốc tế hoạt động phi lợi nhuận, phi chính phủ và bao gồm các thành viên có trình độ chuyên ngành. Tổ chức này chú trọng đến: tiêu chuẩn, giáo dục và các vấn đề về chính sách. Với trên 145 tổ chức thành viên và 65.000 thành viên cá nhân, ISOC bao gồm những con người cụ thể trong cộng đồng Internet. Mọi chi tiết có thể tìm thấy tại website của ISOC.\nInternet Society nằm ở gần thủ đô Washington, DC, Hoa Kỳ và Geneva, Thụy Sĩ. Số hội viên của nó bao gồm hơn 145 tổ chức thành viên và hơn 65.000 cá

In [38]:
for corpus_find in corpus:
    if corpus_find['id'] == "802":
        print(corpus_find)

{'id': '802', 'revid': '4', 'url': 'https://vi.wikipedia.org/wiki?curid=802', 'title': 'Lu-xi-a-na', 'text': ''}
